## Auto mode

This is for testing propouses

In [ ]:
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

In [ ]:
gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
msn_delays=[0,1,2,3,4] # in seconds

## Gazebo Congiguration

In [ ]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

## QGroundControl Configuration

In [ ]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

## No Simulator

In [ ]:
novis_config = ConfigVis[int]()

for i,_ in enumerate(base_homes):
    novis_config.add_vehicle(i)

## Visualization Parameters

In [ ]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [ ]:
simulator = Simulator(
	visualizers=[novis],
	gcs_sysids={'GCS': [1,2,3,4,5]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	verbose=2,
)
orac = simulator.launch()

## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)